In [ ]:
pip install folium

In [ ]:
import pandas as pd
from geopy.distance import geodesic
import folium

# ----------------------------
# Step 1: Load CSV Data
# ----------------------------
cities_data = pd.read_csv("/content/cities in bekaa.csv")  # Ensure it has 'latitude', 'Longitude', 'Location Name in en', 'Pcode'
hospitals_data = pd.read_csv("/content/public hospitals in bekaa.csv")  # Ensure it has 'latitude', 'longitude', 'name', 'hospital_id'

# ----------------------------
# Step 2: Create a Folium Map
# ----------------------------
# Create a base map
m = folium.Map(location=[33.8, 36.0], zoom_start=10)  # Centered around Bekaa

# ----------------------------
# Step 3: Add Hospitals to the Map
# ----------------------------
for _, row in hospitals_data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longtitude']],
        popup=f"Hospital: {row['Name']}",
        icon=folium.Icon(color='green')
    ).add_to(m)

# ----------------------------
# Step 4: Add Cities to the Map
# ----------------------------
for _, row in cities_data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['Longitude']],
        popup=f"City: {row['Location Name in en']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

# ----------------------------
# Step 5: Draw Lines and Distances
# ----------------------------
for _, city in cities_data.iterrows():
    city_coord = (city['latitude'], city['Longitude'])
    for _, hospital in hospitals_data.iterrows():
        hospital_coord = (hospital['latitude'], hospital['longtitude'])
        distance_km = round(geodesic(city_coord, hospital_coord).km, 1)

        # Draw a line between the city and hospital
        folium.PolyLine(locations=[city_coord, hospital_coord], color='black', weight=1).add_to(m)

        # Add distance label at the midpoint
        mid_lat = (city['latitude'] + hospital['latitude']) / 2
        mid_lon = (city['Longitude'] + hospital['longtitude']) / 2
        folium.Marker(
            location=[mid_lat, mid_lon],
            popup=f"{distance_km} km",
            icon=folium.DivIcon(html=f"""<div style="font-size: 10pt">{distance_km} km</div>""")
        ).add_to(m)

# ----------------------------
# Step 6: Save and Show the Map
# ----------------------------
m.save("bekaa_map.html")
m